In [1]:
import sys
import requests
import time
import pickle
from pprint import pprint

In [2]:
def pickle_stuff(filename, data):
    ''' save file '''
    with open(filename, 'w') as picklefile:
        pickle.dump(data, picklefile)

def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [2]:
with open("echonestapi2.txt", 'r') as ena:
    api_key = ena.read().strip()

In [3]:
rate_time = 0.501 #3.1

In [3]:
r = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+'&id=musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d')

In [4]:
rdata = r.json()
pprint(rdata)

{u'response': {u'status': {u'code': 0,
                           u'message': u'Success',
                           u'version': u'4.2'},
               u'track': {u'artist': u'Hammock',
                          u'catalog': u'musicbrainz',
                          u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
                          u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
                          u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
                          u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
                          u'id': u'TRRYMSJ13B7BF722BF',
                          u'release': u'More Dead Than Alive (Get Away From the Medicine)',
                          u'song_id': u'SOCGKJR12A58A79D30',
                          u'status': u'complete',
                          u'title': u'I Can Almost See You'}}}


In [5]:
pprint(rdata["response"]["status"]["message"])

u'Success'


In [8]:
pprint(rdata["response"]["track"])
pprint(rdata["response"]["track"]["song_id"])

{u'artist': u'Hammock',
 u'catalog': u'musicbrainz',
 u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
 u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
 u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
 u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
 u'id': u'TRRYMSJ13B7BF722BF',
 u'release': u'More Dead Than Alive (Get Away From the Medicine)',
 u'song_id': u'SOCGKJR12A58A79D30',
 u'status': u'complete',
 u'title': u'I Can Almost See You'}
u'SOCGKJR12A58A79D30'


In [117]:
skiplines = 8900

In [118]:
mbid_songs2 = {}

In [119]:
with open("mbid_songs_part1000.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs2[key] = track

In [120]:
print(len(mbid_songs2.keys()))

1168899


In [121]:
linecount = 0
with open("mbids.txt", 'r') as mf:
    for s in xrange(skiplines):
        next(mf)
    t0 = time.time()
    for line in mf:
        linecount += 1
        print len(line.split())
        t1 = time.time()
        count = 0
        for mbid in line.split():
            if mbid in mbid_songs2:
                #print "continuing"
                count += 1
                continue
            print count,
            count += 1
            moveon = False
            while not moveon:
                #print "requesting"
                time.sleep(rate_time)
                try:
                    enr = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+
                                       '&id=musicbrainz:track:'+mbid)
                    enrj = enr.json()
                    status = enrj["response"]["status"]["message"]
                    if status == "Success":
                        #print "true"
                        if "song_id" in enrj["response"]["track"]:
                            #print "success" #"ids", mbid
                            mbid_songs2[mbid] = enrj["response"]["track"]["song_id"]
                        moveon = True
                    elif status == "The Identifier specified does not exist":
                        #echo nest does not have all mbids mapped to tracks
                        #print status #mbid, status
                        mbid_songs2[mbid] = ""
                        moveon = True
                    else:
                        print status
                        time.sleep(3)
                except KeyboardInterrupt:
                    raise
                except: # catch *all* exceptions
                    e = sys.exc_info()[0]
                    print e,
                    time.sleep(3)
        t2 = time.time()
        if t2-t1 > 0.5:
            print "\nline seconds:", t2-t1,
            print "total seconds:", t2-t0, 
            print "total lines:", linecount     

682
737
3 20 25 39 43 46 49 65 107 125 180 194 239 240 244 246 311 365 394 406 517 518 531 555 572 589 598 678 704 731 
line seconds: 26.4959499836 total seconds: 26.4967961311 total lines: 2
811
22 36 72 96 108 122 142 149 155 174 193 197 221 317 351 358 360 363 373 383 403 404 407 411 445 451 476 489 505 513 545 548 599 657 671 677 689 690 737 744 766 779 784 800 
line seconds: 38.2796299458 total seconds: 64.7769770622 total lines: 3
517
268 371 445 
line seconds: 2.44974207878 total seconds: 67.2272071838 total lines: 4
0
777
17 43 71 73 78 135 140 172 173 201 228 241 264 293 316 354 359 405 462 469 473 484 500 502 516 542 555 556 572 580 593 648 652 668 683 728 
line seconds: 42.6509859562 total seconds: 109.878929138 total lines: 6
1225
2 4 11 19 73 77 107 109 151 153 171 190 210 215 246 251 287 356 369 370 386 450 463 481 497 501 550 559 591 601 610 659 709 782 824 834 846 862 910 913 962 995 1002 1100 1105 1131 1167 1169 1172 1192 1196 1205 1209 1214 
line seconds: 45.298758983

In [124]:
print len(mbid_songs2.keys())

1181150


In [125]:
with open("mbid_songs_part1000.csv", 'w') as msf:
    for k, v in mbid_songs2.iteritems():
        msf.write(k+","+v+"\n")

In [23]:
#pickle_stuff("mbid_song_dict_part1000.pkl", mbid_songs2)

In [108]:
#print mbid_songs